In [81]:
URL ="https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max=2016-11-24&date_received_min=2016-10-23&field=all&format=json"

In [82]:
import json
import requests

In [83]:

data = requests.get(URL)


In [84]:
with open("../filtered_data.json", "w") as file_obj:
    finance_complaint_data = list(map(lambda x: x["_source"],
                                                json.loads(data.content)))
                                    

    json.dump(finance_complaint_data, file_obj)


In [85]:
import pandas as pd

df=pd.read_json("../filtered_data.json")

df.head()

,product,complaint_what_happened,date_sent_to_company,issue,sub_product,zip_code,tags,complaint_id,timely,consumer_consent_provided,company_response,submitted_via,company,date_received,state,consumer_disputed,company_public_response,sub_issue
0,Bank account or service,"TRANSACTIONS, NONE. I did this XX/XX/XXXX afte...",2016-11-14T12:00:00-05:00,Problems caused by my funds being low,Checking account,20011.0,Servicemember,2206373,Yes,Consent provided,Closed with explanation,Web,M&T BANK CORPORATION,2016-11-14T12:00:00-05:00,DC,Yes,Company has responded to the consumer and the ...,None
1,Mortgage,,2016-12-20T12:00:00-05:00,"Loan modification,collection,foreclosure",Other mortgage,71832.0,Servicemember,2210501,Yes,N/A,Closed with explanation,Referral,HSBC NORTH AMERICA HOLDINGS INC.,2016-11-16T12:00:00-05:00,AR,No,None,None
2,Debt collection,,2016-11-12T12:00:00-05:00,Cont'd attempts collect debt not owed,I do not know,30324.0,None,2195801,Yes,Consent not provided,Closed with explanation,Web,Avant Credit Corporation,2016-11-05T12:00:00-05:00,GA,No,None,Debt resulted from identity theft
3,Debt collection,I WENT THRU SMALL CLAIMS COURT IN XXXX AND SUE...,2016-11-18T12:00:00-05:00,Cont'd attempts collect debt not owed,"Other (i.e. phone, health club, etc.)",94590.0,None,2214010,No,Consent provided,Closed,Web,"Lazarus Financial Group, Inc.(Closed)",2016-11-17T12:00:00-05:00,CA,No,Company believes complaint is the result of an...,Debt is not mine
4,Mortgage,Wells Fargo Home Mortgage continues to collect...,2016-10-23T12:00:00-05:00,"Loan servicing, payments, escrow account",VA mortgage,76504.0,Older American,2174131,Yes,Consent provided,Closed with explanation,Web,WELLS FARGO & COMPANY,2016-10-23T12:00:00-05:00,TX,No,Company has responded to the consumer and the ...,None


In [86]:
df.shape

(17591, 18)

In [87]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col


In [ ]:
# spark.sparkContext.stop()

In [88]:
spark = SparkSession.builder.master('local[*]').appName('finance_complaint')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()

In [89]:
sql = spark.sql

In [90]:
spark.sparkContext._conf.getAll()

[('spark.app.startTime', '1669698904050'),
 ('spark.sql.warehouse.dir',
  'file:/config/workspace/Notebook/spark-warehouse'),
 ('spark.driver.port', '36299'),
 ('spark.app.id', 'local-1669698905482'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '6g'),
 ('spark.app.name', 'finance_complaint'),
 ('spark.executor.memoryOverhead', '8g'),
 ('spark.driver.memory', '6g'),
 ('spark.driver.host', 'c783cf6aa7ee'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.executor.instances', '1'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [99]:
df = spark.read.json("../filtered_data.json")

In [100]:
df.count()

17591

In [101]:
df.show()

+--------------------+-----------------------+--------------------+------------+-----------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+-------------+--------------+------+--------+
|             company|company_public_response|    company_response|complaint_id|complaint_what_happened|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|         sub_product|submitted_via|          tags|timely|zip_code|
+--------------------+-----------------------+--------------------+------------+-----------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+-------------+--------------+------+--------+
|M&T BANK CORPORATION|

In [102]:
df.write.mode('append').parquet("../complaint_data")

In [103]:
df=spark.read.parquet("../complaint_data")

In [104]:
df.count()

17591

In [105]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [106]:
df.select('complaint_id')

DataFrame[complaint_id: string]

In [107]:
## print unique values

for column in df.columns:
    print(f"{column}:{df.select(column).distinct().count()}")

company:1256
company_public_response:9
company_response:5
complaint_id:17591
complaint_what_happened:7020
consumer_consent_provided:5
consumer_disputed:2
date_received:33
date_sent_to_company:114
issue:86
product:12
state:58
sub_issue:58
sub_product:45
submitted_via:6
tags:4
timely:2
zip_code:7239


In [108]:
df.describe(["timely"]).show()

+-------+------+
|summary|timely|
+-------+------+
|  count| 17591|
|   mean|  null|
| stddev|  null|
|    min|    No|
|    max|   Yes|
+-------+------+



In [109]:
for col in df.columns:
    df.describe([col]).show()

+-------+--------------------+
|summary|             company|
+-------+--------------------+
|  count|               17591|
|   mean|                null|
| stddev|                null|
|    min|1 STOP MONEY CENT...|
|    max|iFreedom Direct C...|
+-------+--------------------+

+-------+-----------------------+
|summary|company_public_response|
+-------+-----------------------+
|  count|                   9178|
|   mean|                   null|
| stddev|                   null|
|    min|   Company believes ...|
|    max|   Company has respo...|
+-------+-----------------------+

+-------+-----------------+
|summary| company_response|
+-------+-----------------+
|  count|            17591|
|   mean|             null|
| stddev|             null|
|    min|           Closed|
|    max|Untimely response|
+-------+-----------------+

+-------+------------------+
|summary|      complaint_id|
+-------+------------------+
|  count|             17591|
|   mean|  2198419.29844807|
| stddev|13938.

In [110]:
for col in df.columns:
    print(col,  df.filter(df[col].isNull()).count())

company 0
company_public_response 8413
company_response 0
complaint_id 0
complaint_what_happened 0
consumer_consent_provided 1
consumer_disputed 0
date_received 0
date_sent_to_company 0
issue 0
product 0
state 119
sub_issue 9099
sub_product 6119
submitted_via 0
tags 14949
timely 0
zip_code 118


In [53]:
df.groupBy(df.company_response).count().show()
df.groupBy(df.consumer_consent_provided).count().show()
df.groupBy(df.consumer_disputed).count().show()
df.groupBy(df.product).count().show()
df.groupBy(df.submitted_via).count().show()
df.groupBy(df.timely).count().show()

+--------------------+-----+
|    company_response|count|
+--------------------+-----+
|   Untimely response|   94|
|Closed with non-m...| 3845|
|         In progress|48984|
|Closed with monet...|  450|
|Closed with expla...|18373|
+--------------------+-----+

+-------------------------+-----+
|consumer_consent_provided|count|
+-------------------------+-----+
|                     null|59287|
|        Consent withdrawn|  186|
|                    Other| 2850|
|         Consent provided| 3206|
|     Consent not provided| 4079|
|                      N/A| 2138|
+-------------------------+-----+

+-----------------+-----+
|consumer_disputed|count|
+-----------------+-----+
|              N/A|71746|
+-----------------+-----+

+--------------------+-----+
|             product|count|
+--------------------+-----+
|     Debt collection| 3272|
|Money transfer, v...|  815|
|Checking or savin...| 2235|
|            Mortgage| 1104|
|Credit card or pr...| 2464|
|Credit reporting,...|60332|
|Vehi

In [111]:
df.groupBy(df.product).count().collect()

[Row(product='Debt collection', count=3804),
 Row(product='Virtual currency', count=1),
 Row(product='Payday loan', count=153),
 Row(product='Money transfers', count=149),
 Row(product='Mortgage', count=3631),
 Row(product='Prepaid card', count=94),
 Row(product='Credit reporting', count=3941),
 Row(product='Consumer Loan', count=897),
 Row(product='Credit card', count=2025),
 Row(product='Bank account or service', count=2099),
 Row(product='Other financial service', count=50),
 Row(product='Student loan', count=747)]

In [112]:
df.company_response

Column<'company_response'>

In [113]:
def update_column_attribute(df):
    for column in df.columns:
        setattr(df,column,column)

In [114]:
update_column_attribute(df)

In [115]:
df.company_response

'company_response'

In [116]:
df.count()

17591

In [117]:
complaint_table="complaint"
df.createOrReplaceTempView(complaint_table)

In [118]:
missing_target_df = sql(f"select * from {complaint_table} where {df.consumer_disputed} ='N/A' ")

In [119]:
df = sql(f"select * from {complaint_table} where {df.consumer_disputed} <>'N/A' ")

In [120]:
df.count()

17591

In [121]:
complaint_table="complaint2"
df.createOrReplaceTempView(complaint_table)

In [122]:
update_column_attribute(df)

In [123]:
def perform_null_analysis(df,table_name):
    null_value_analysis=[]
    n_row=df.count()
    for column in df.columns:
        print(column)
        response = sql(f"select {n_row} as  total_row,count(*) as null_row_{column},(count(*)*100)/{n_row} as missing_percentage,'{column}' as  column_name from {table_name} where {column} is null").collect()
        null_value_analysis.append(response)
    return null_value_analysis

In [124]:
null_report = perform_null_analysis(df,complaint_table)

company
company_public_response
company_response
complaint_id
complaint_what_happened
consumer_consent_provided
consumer_disputed
date_received
date_sent_to_company
issue
product
state
sub_issue
sub_product
submitted_via
tags
timely
zip_code


In [125]:
null_report

[[Row(total_row=17591, null_row_company=0, missing_percentage=0.0, column_name='company')],
 [Row(total_row=17591, null_row_company_public_response=8413, missing_percentage=47.82559263259621, column_name='company_public_response')],
 [Row(total_row=17591, null_row_company_response=0, missing_percentage=0.0, column_name='company_response')],
 [Row(total_row=17591, null_row_complaint_id=0, missing_percentage=0.0, column_name='complaint_id')],
 [Row(total_row=17591, null_row_complaint_what_happened=0, missing_percentage=0.0, column_name='complaint_what_happened')],
 [Row(total_row=17591, null_row_consumer_consent_provided=1, missing_percentage=0.00568472514353931, column_name='consumer_consent_provided')],
 [Row(total_row=17591, null_row_consumer_disputed=0, missing_percentage=0.0, column_name='consumer_disputed')],
 [Row(total_row=17591, null_row_date_received=0, missing_percentage=0.0, column_name='date_received')],
 [Row(total_row=17591, null_row_date_sent_to_company=0, missing_percent

In [126]:
def unwanted_column_by_missing_percentage(null_value_analysis,per_thres=20):
    columns= []
    for row in null_value_analysis:
        row_info=row[0]
        if row_info.missing_percentage>per_thres:
            print(row_info)
            columns.append(row_info.column_name)

    return columns


In [127]:
columns = unwanted_column_by_missing_percentage(null_value_analysis=null_report)

Row(total_row=17591, null_row_company_public_response=8413, missing_percentage=47.82559263259621, column_name='company_public_response')
Row(total_row=17591, null_row_sub_issue=9099, missing_percentage=51.725314081064184, column_name='sub_issue')
Row(total_row=17591, null_row_sub_product=6119, missing_percentage=34.78483315331704, column_name='sub_product')
Row(total_row=17591, null_row_tags=14949, missing_percentage=84.98095617076915, column_name='tags')


In [128]:
def drop_column(df,columns):
    selected_column = list(filter(lambda x:x not in columns,df.columns))
    selected_column = ",".join(selected_column)
    df= sql(f"select {selected_column}  from {complaint_table} ")
    return df

In [129]:
df=drop_column(df,columns)

In [130]:
columns = perform_null_analysis(df,complaint_table)

company
company_response
complaint_id
complaint_what_happened
consumer_consent_provided
consumer_disputed
date_received
date_sent_to_company
issue
product
state
submitted_via
timely
zip_code


In [131]:
unwanted_column_by_missing_percentage(columns)

[]

In [134]:
update_column_attribute(df)

In [135]:
df=drop_column(df,columns=[df.complaint_id])

In [170]:
update_column_attribute(df)

In [139]:
ONE_HOT_FEATURE = [df.company_response,df.consumer_consent_provided,df.submitted_via,df.timely]
BINARY_ENCODING = [df.product,]
TARGET_FEATURE = [df.consumer_disputed]

#df.company_response No null value
#df.consumer_consent_provided  replace null with top category 
#df.consumer_disputed target feature label encoding
#df.product one hot encoding 

#df.product no null value

In [140]:
for i in ONE_HOT_FEATURE:
    print(i)

company_response
consumer_consent_provided
submitted_via
timely


In [141]:
remaining_column = list(filter(lambda x: x not in ONE_HOT_FEATURE+BINARY_ENCODING+TARGET_FEATURE,df.columns))

In [142]:
remaining_column

['company',
 'complaint_what_happened',
 'date_received',
 'date_sent_to_company',
 'issue',
 'state',
 'zip_code']

In [143]:
df.groupBy(df.company).count().count()

1256

In [144]:
FREQUENCY_ENCODING = [df.company]

In [145]:
df.select(df.company).count()-df.select(df.company).dropna().count()

0

In [146]:
REPLACE_NULL_WITH_TOP_VALUE = [df.zip_code,df.state,df.consumer_consent_provided]

In [147]:
REPLACE_NULL_WITH_TOP_VALUE

['zip_code', 'state', 'consumer_consent_provided']

In [148]:
print(f"Total number of row: {df.count()}")
for column in remaining_column:
    print(f"{column}:  {df.select(column).distinct().count()}")

Total number of row: 17591
company:  1256
complaint_what_happened:  7020
date_received:  33
date_sent_to_company:  114
issue:  86
state:  58
zip_code:  7239


In [149]:
df.groupBy(df.issue).count().show()

+--------------------+-----+
|               issue|count|
+--------------------+-----+
|Communication tac...|  434|
|Application proce...|   18|
|Advertising and m...|   70|
|Balance transfer fee|    6|
|Customer service/...|   15|
|        Adding money|    3|
|Credit card prote...|   60|
|Closing/Cancellin...|  150|
|Received a loan I...|   14|
|          Bankruptcy|   12|
|                Fees|    3|
|Can't stop charge...|   17|
|Forbearance / Wor...|    8|
|Credit determination|   61|
|Loan modification...| 1330|
|    Cash advance fee|    6|
|Other transaction...|   49|
|Customer service ...|   71|
|      Getting a loan|   11|
|  Delinquent account|   72|
+--------------------+-----+
only showing top 20 rows



In [150]:
sql(f"select {df.complaint_what_happened} from {complaint_table} limit 5").collect()

[Row(complaint_what_happened="TRANSACTIONS, NONE. I did this XX/XX/XXXX after M & T bank beat me for {$80.00} on a technicality. Despite having definitively placing my account in a no overdraft protection status. M & T has once again paid something on my account and are attempting to justify charging me nsf fees for the overdraft by claiming this as a recurring payment versus a one time payment. Which it is n't and never has been. I pay this manually based on the months that I want the service, if you notice on the account for this vendor, it 's paid on different dates of the months that I use the service. When I placed my account in a status of declining overdraft protection, M & T bank did n't tell me I needed to specify transaction types in which I wanted no overdraft protection. No overdraft means no overdraft, does n't matter the type of transaction."),
 Row(complaint_what_happened=''),
 Row(complaint_what_happened=''),
 Row(complaint_what_happened="I WENT THRU SMALL CLAIMS COURT 

In [151]:
TOKENIZER_FEATURE = [df.complaint_what_happened]

In [153]:
FREQUENCY_ENCODING = [df.company,df.issue,df.state,df.zip_code]

In [154]:
TARGET_FEATURE

['consumer_disputed']

In [155]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col


In [171]:
df=df.withColumn(df.date_received,col(df.date_received).cast(TimestampType()))

In [173]:
update_column_attribute(df)

In [174]:
df=df.withColumn(df.date_sent_to_company,col(df.date_sent_to_company).cast(TimestampType()))
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: timestamp (nullable = true)
 |-- date_sent_to_company: timestamp (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [175]:
df.select([df.date_received,df.date_sent_to_company]).show()

+-------------------+--------------------+
|      date_received|date_sent_to_company|
+-------------------+--------------------+
|2016-11-14 22:30:00| 2016-11-14 22:30:00|
|2016-11-16 22:30:00| 2016-12-20 22:30:00|
|2016-11-05 22:30:00| 2016-11-12 22:30:00|
|2016-11-17 22:30:00| 2016-11-18 22:30:00|
|2016-10-23 22:30:00| 2016-10-23 22:30:00|
|2016-10-31 22:30:00| 2016-10-31 22:30:00|
|2016-11-11 22:30:00| 2016-11-16 22:30:00|
|2016-10-26 22:30:00| 2016-10-26 22:30:00|
|2016-11-09 22:30:00| 2016-11-10 22:30:00|
|2016-10-23 22:30:00| 2017-01-06 22:30:00|
|2016-11-01 22:30:00| 2016-11-14 22:30:00|
|2016-11-17 22:30:00| 2016-11-17 22:30:00|
|2016-11-14 22:30:00| 2016-11-15 22:30:00|
|2016-11-05 22:30:00| 2016-11-05 22:30:00|
|2016-11-02 22:30:00| 2016-11-02 22:30:00|
|2016-11-09 22:30:00| 2016-11-27 22:30:00|
|2016-11-15 22:30:00| 2016-11-15 22:30:00|
|2016-11-02 22:30:00| 2016-12-09 22:30:00|
|2016-11-01 22:30:00| 2016-11-01 22:30:00|
|2016-11-10 22:30:00| 2016-11-10 22:30:00|
+----------

In [178]:
update_column_attribute(df)

In [176]:
from pyspark.sql.types import  LongType

In [179]:
df = df.withColumn("diff_in_days",(col(df.date_sent_to_company).cast(LongType())-col(df.date_received).cast(LongType()))/(60*60*24))

In [180]:
update_column_attribute(df)

In [182]:
df=df.drop(col(df.date_received)).drop(col(df.date_sent_to_company))

In [183]:
NUMERICAL_FEATURE = [df.diff_in_days,]
ONE_HOT_FEATURE+\
FREQUENCY_ENCODING+\
BINARY_ENCODING+\
TARGET_FEATURE

['company_response',
 'consumer_consent_provided',
 'submitted_via',
 'timely',
 'company',
 'issue',
 'state',
 'zip_code',
 'product',
 'consumer_disputed']